In [1]:
# coding: utf-8
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

import os
import random

In [2]:
negative = [f for f in os.listdir('./review_polarity 2/txt_sentoken/neg') if f.endswith('.txt')]

In [3]:
positive = [f for f in os.listdir('./review_polarity 2/txt_sentoken/pos') if f.endswith('.txt')]

In [21]:
arr_reviews = []

for review in negative:
    with open('./review_polarity 2/txt_sentoken/neg/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '0'
        dicti['text'] = text
        arr_reviews.append(dicti)
        
for review in positive:
    with open('./review_polarity 2/txt_sentoken/pos/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '1'
        dicti['text'] = text
        arr_reviews.append(dicti)

        
random.shuffle(arr_reviews)  # перемешиваем данные
    
df = pd.DataFrame(arr_reviews)  # создаем датасет


In [5]:
df.head()

,class,text
0,0,wizards is an animated feature that begins wit...
1,1,""" he's back , and it's about time . "" \nwas t..."
2,1,review- peter jackson's the frighteners has re...
3,1,"i like movies with albert brooks , and i reall..."
4,1,contact ( pg ) there's a moment late in robert...


In [6]:
df.describe()

,class,text
count,2000,2000
unique,2,2000
top,0,apollo 13 is simply one of the most dramatic f...
freq,1000,1


In [7]:
df.groupby('class').describe()

text
class                                                          
0     count                                                1000
      unique                                               1000
      top     when the mediums in question are video game an...
      freq                                                    1
1     count                                                1000
      unique                                               1000
      top     apollo 13 is simply one of the most dramatic f...
      freq                                                    1

In [8]:
#  Делим выборку в соотновении 80:20
rev_train, rev_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [9]:
rev_train.shape, rev_test.shape, class_train.shape, class_test.shape

((1600,), (400,), (1600,), (400,))

In [20]:
# Векторизация Bag of Words
bow = CountVectorizer()
bow.fit_transform(rev_train)
bowed_rev_train = bow.transform(rev_train)  # тренировочные тексты
bowed_rev_test = bow.transform(rev_test)   # тестовые тексты

In [11]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(bowed_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.48      1.00      0.65       194
          1       0.00      0.00      0.00       206

avg / total       0.24      0.48      0.32       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(bowed_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [13]:
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.83      0.89      0.86       194
          1       0.89      0.83      0.85       206

avg / total       0.86      0.85      0.85       400



In [14]:
# Векторизация TF-IDF
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.8,
                             sublinear_tf=True,
                             use_idf=True)
tfidf_rev_train = vectorizer.fit_transform(rev_train)
tfidf_rev_test = vectorizer.transform(rev_test)

In [19]:
# Обучение DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(tfidf_rev_train, class_train)

# Вывод результатов по Dummy Classifier
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.48      1.00      0.65       194
          1       0.00      0.00      0.00       206

avg / total       0.24      0.48      0.32       400



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Наша модель справилась с определением сентимента лучше, чем DummyClassifier

In [16]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(tfidf_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [17]:
# Печатаем результаты классификации
print(classification_report(class_test, clf.predict(tfidf_rev_test)))

             precision    recall  f1-score   support

          0       0.86      0.88      0.87       194
          1       0.89      0.86      0.87       206

avg / total       0.87      0.87      0.87       400



Наша модель справилась с определением сентимента лучше, чем DummyClassifier